In [1]:
import pandas as pd

X=pd.read_csv('train.csv')
X_test_full=pd.read_csv('test.csv')

Same, dropping rows with missing values at target var and then separating

In [2]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [3]:
y=X.SalePrice

In [4]:
X.drop(['SalePrice'], axis=1, inplace=True)

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid=train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

In [6]:
# calulating cardinality and selecting columns with low cardinality and type object

low_card_cols=[cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype =='object']

In [7]:
#collecting numerical columns

In [8]:
numerical_cols=[cname for cname in X_train.columns if X_train[cname].dtype in ['int64','float64']]

In [9]:
# keeping only selected columns

In [10]:
my_columns=low_card_cols + numerical_cols

In [11]:
X_train_real=X_train[my_columns].copy()
X_valid_real=X_valid[my_columns].copy()
X_test=X_test_full[my_columns].copy()

In [12]:
# now we need to implement one hot encoder

It can be done in pandas or with function

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [14]:
numerical_trans=SimpleImputer(strategy='constant')

In [15]:
categorical_trans=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [16]:
preprocessor=ColumnTransformer(transformers=[
    ('num', numerical_trans, numerical_cols),
    ('cat', categorical_trans, low_card_cols)
])

Building a model

I will implement one hot encoder using pipeline

In [17]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [18]:
my_model_1=XGBRegressor(random_state=0)

In [19]:
my_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', my_model_1)
])
my_pipeline.fit(X_train,y_train)
pred=my_pipeline.predict(X_valid)
score=mean_absolute_error(y_valid, pred)
print('MAE :')
print(score)

C:\Users\Hana\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


MAE :
16748.130190496577


Can we improve model??

In [20]:
my_model_2=XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_pipeline2=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', my_model_2)
])
my_pipeline2.fit(X_train,y_train)
pred2=my_pipeline2.predict(X_valid)
score2=mean_absolute_error(y_valid, pred2)
print('MAE :')
print(score2)

C:\Users\Hana\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


MAE :
16700.213733411816


Can we break model?

We know that used estimators are about 100-1000 but if we put 1 will there must be underfitting!?

In [22]:
my_model_3=XGBRegressor(n_estimators=1)
my_pipeline3=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', my_model_3)
])
my_pipeline3.fit(X_train,y_train)
pred3=my_pipeline3.predict(X_valid)
score3=mean_absolute_error(y_valid, pred3)
print('MAE :')
print(score3)

MAE :
163382.59044574058


C:\Users\Hana\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Yes! We got bigger mean absoulute error which means that we have under fitting. 